<h1>Оглавление<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Изучение-общей-информации" data-toc-modified-id="Изучение-общей-информации-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Изучение общей информации</a></span></li><li><span><a href="#Предобработка-данных" data-toc-modified-id="Предобработка-данных-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Предобработка данных</a></span><ul class="toc-item"><li><span><a href="#Обработка-пропусков" data-toc-modified-id="Обработка-пропусков-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Обработка пропусков</a></span></li><li><span><a href="#Замена-типа-данных" data-toc-modified-id="Замена-типа-данных-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Замена типа данных</a></span></li><li><span><a href="#Обработка-дубликатов" data-toc-modified-id="Обработка-дубликатов-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Обработка дубликатов</a></span></li><li><span><a href="#Лемматизация" data-toc-modified-id="Лемматизация-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Лемматизация</a></span></li><li><span><a href="#Категоризация-данных" data-toc-modified-id="Категоризация-данных-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>Категоризация данных</a></span></li></ul></li><li><span><a href="#Ответим-на-вопросы:" data-toc-modified-id="Ответим-на-вопросы:-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Ответим на вопросы:</a></span></li><li><span><a href="#Общий-вывод" data-toc-modified-id="Общий-вывод-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Общий вывод</a></span></li></ul></div>

# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Изучение общей информации

In [1]:
import pandas as pd # импорт библиотеки pandas
bc = pd.read_csv('/datasets/data.csv') # чтение файла с данными и сохранение в bc (bank_customers или bank_clients)
bc.head(10) # получение первых 10 строк таблицы

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [2]:
bc.info() # получение общей информации о данных в таблице

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Вывод**

В таблице 12 столбцов и 21525 строк.

Столбцы содержат типы данных `float, int, oblect`. Наименования столбцов корректны и единообразны.

При первоначальном ознакомлении с таблицей и информацией о ней обнаружены следующие проблемы: 
- отрицательные показатели в столбце `days_employed`. В нём содержится количество дней трудового стажа, которое не может быть меньше ноля - соответственно, минусы перед числом, скорее всего, являются технической ошибкой, и их можно отбросить;
- отсутствие единообразия в содержании столбца `education` - данные зафиксированы разным регистром, что может спровоцировать проблемы в дальнейшей работе с таблицей (например, при попытке выявления явных дубликатов);
- наличие пропусков в столбцах `days_employed` и `total_income`; 
- информация в столбцах `days_employed` и `total_income` имеет тип данных `float` с шестью знаками после запятой - они не содержат принципиально важной и полезной информации и усложняют визуальную работу с таблицей.


Все перечисленные проблемы будем решать на этапе предобработки данных.

## Предобработка данных

### Обработка пропусков

При знакомстве с информацией о таблице нами обнаружено, что в столбцах `days_employed` и `total_income` имеются пропущенные значения. Но прежде чем заняться заменой пропусков, отработаем артефакты в столбце `days_employed`.

In [3]:
bc['days_employed'] = bc['days_employed'].abs() # получение модуля числа и сохранение в столбце days_employed
bc.head() # получение первых 5 строк таблицы

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


Теперь, когда в нашей таблице нет значений, не отражающих действительность, можно приступить к работе с пропусками.

Итак, столбец `days_employed` содержит в себе информацию об общем трудовом стаже в днях, в столбце `total_income` зафиксированы данные о ежемесячном доходе клиентов. Учитывая, что перед нами статистика от кредитного отдела банка, вряд ли пропуски связаны с личным нежеланием клиентов указывать о себе эту информацию, так как она является обязательной для запроса кредита. По этой же причине очевидно, что отсутствующие значения не значат "0". К тому же подобные пропуски есть в строках с информацией о взрослых и пожилых клиентах, чей трудовой стаж не может быть равен нулю. 
Можно предположить, что данные о стаже и ежемесячном доходе отсутствуют в тех случаях, когда они не подтверждены документально (трудовой книжкой, справкой 2-НДФЛ или выпиской из банка). 

Таким образом, в обоих столбцах мы имеем дело с количественными переменными. Поэтому пропуски будем заменять медианными значениями, как наиболее объективными в сравнении со средними значениями.

In [4]:
bc['days_employed'] = bc['days_employed'].fillna(bc['days_employed'].median()) # замена пропусков в days_employed на медианное значение по столбцу
bc ['total_income'] = bc['total_income'].fillna(bc['total_income'].median()) # замена пропусков в total_income на медианное значение по столбцу
bc.info() # получение информации об обновленной таблице для проверки

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Вывод**

Теперь в нашей таблице нет пропусков и артефактов, но остались цифры с длинными и ненужными "хвостами" после запятой. Нам необходимо преобразовать `float` в `int`, поэтому будем использовать метод `astype()`.

### Замена типа данных

In [5]:
bc[['days_employed', 'total_income']] = bc[['days_employed', 'total_income']].astype(int) # изменение типа данных на int в столбцах days_employed и total_income
bc.info() # получение информации об обновленной таблице для проверки

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


**Вывод**

Согласно полученной информации о таблице, теперь все столбцы имеют типы `int` и `object` - наша задача по замене `float` выполнена.

### Обработка дубликатов

В процессе знакомства с таблицой мы обнаружили, что в столбце `education` данные зафиксированы разным регистром. Этот факт может помешать нам при выявлении явных дубликатов, поэтому приведём все значения в столбце к единому строчному регистру. 

In [6]:
bc['education'] = bc['education'].str.lower() # приведение строк в столбце education к нижнему регистру

Теперь, когда все строки в колонке `education` приведены к единому регистру, информация о наличии дубликатов будет более корректна. Метод `duplicated()` укажет нам только на наличие либо отсутствие дубликатов, поэтому посчитаем общее количество имеющихся дубликатов, добавив метод `sum()`.

In [7]:
bc.duplicated().sum() # подсчёт явных дубликатов

71

71 дубликат из 21525 - не так много, удалим их методом `drop_duplicates()`, также удалив старые индексы и сформировав новые.

In [8]:
bc = bc.drop_duplicates().reset_index(drop=True) # удаление явных дубликатов (с удалением старых индексов и формированием новых)
bc.duplicated().sum() # проверка на отсутствие дубликатов

0

**Вывод**

Дубликаты удалены. Их возникновение может быть связано с подачей одним клиентом заявлений в несколько отделений банка либо же с какими-то техническими причинами, ошибками.

### Лемматизация

In [9]:
from pymystem3 import Mystem
m = Mystem() # импорт библиотеки pymystem3 
purposes = ' '.join(bc['purpose'].unique()) 
# получение уникальных значений столбца purpose, объединение их в строку методом join(), сохранение в переменную purposes
purposes

'покупка жилья приобретение автомобиля дополнительное образование сыграть свадьбу операции с жильем образование на проведение свадьбы покупка жилья для семьи покупка недвижимости покупка коммерческой недвижимости покупка жилой недвижимости строительство собственной недвижимости недвижимость строительство недвижимости на покупку подержанного автомобиля на покупку своего автомобиля операции с коммерческой недвижимостью строительство жилой недвижимости жилье операции со своей недвижимостью автомобили заняться образованием сделка с подержанным автомобилем получение образования автомобиль свадьба получение дополнительного образования покупка своего жилья операции с недвижимостью получение высшего образования свой автомобиль сделка с автомобилем профильное образование высшее образование покупка жилья для сдачи на покупку автомобиля ремонт жилью заняться высшим образованием'

In [10]:
lemmas = m.lemmatize(purposes) # лемматизация слов в строке purposes
lemmas # выведение лемм на экран

['покупка',
 ' ',
 'жилье',
 ' ',
 'приобретение',
 ' ',
 'автомобиль',
 ' ',
 'дополнительный',
 ' ',
 'образование',
 ' ',
 'сыграть',
 ' ',
 'свадьба',
 ' ',
 'операция',
 ' ',
 'с',
 ' ',
 'жилье',
 ' ',
 'образование',
 ' ',
 'на',
 ' ',
 'проведение',
 ' ',
 'свадьба',
 ' ',
 'покупка',
 ' ',
 'жилье',
 ' ',
 'для',
 ' ',
 'семья',
 ' ',
 'покупка',
 ' ',
 'недвижимость',
 ' ',
 'покупка',
 ' ',
 'коммерческий',
 ' ',
 'недвижимость',
 ' ',
 'покупка',
 ' ',
 'жилой',
 ' ',
 'недвижимость',
 ' ',
 'строительство',
 ' ',
 'собственный',
 ' ',
 'недвижимость',
 ' ',
 'недвижимость',
 ' ',
 'строительство',
 ' ',
 'недвижимость',
 ' ',
 'на',
 ' ',
 'покупка',
 ' ',
 'подержать',
 ' ',
 'автомобиль',
 ' ',
 'на',
 ' ',
 'покупка',
 ' ',
 'свой',
 ' ',
 'автомобиль',
 ' ',
 'операция',
 ' ',
 'с',
 ' ',
 'коммерческий',
 ' ',
 'недвижимость',
 ' ',
 'строительство',
 ' ',
 'жилой',
 ' ',
 'недвижимость',
 ' ',
 'жилье',
 ' ',
 'операция',
 ' ',
 'со',
 ' ',
 'свой',
 ' ',
 'недвижимо

In [11]:
from collections import Counter # импорт контейнера Counter из модуля collections
print(Counter(lemmas)) # подсчет числа упоминаний лемм

Counter({' ': 96, 'покупка': 10, 'недвижимость': 10, 'автомобиль': 9, 'образование': 9, 'жилье': 7, 'с': 5, 'операция': 4, 'на': 4, 'свой': 4, 'свадьба': 3, 'строительство': 3, 'получение': 3, 'высокий': 3, 'дополнительный': 2, 'для': 2, 'коммерческий': 2, 'жилой': 2, 'подержать': 2, 'заниматься': 2, 'сделка': 2, 'приобретение': 1, 'сыграть': 1, 'проведение': 1, 'семья': 1, 'собственный': 1, 'со': 1, 'профильный': 1, 'сдача': 1, 'ремонт': 1, '\n': 1})


**Вывод**

По результатам лемматизации слов в колонке purpose мы можем определить, что цели, указанные для получения кредита, делятся на четыре основные группы: 
- операции с недвижимостью (жилой/коммерческой) - покупка/ремонт/строительство,
- приобретение автомобиля,
- траты на образование,
- расходы на свадьбу.

[Четвертый вопрос исследования](#4?) запрашивает наличие зависимости между целями кредита и возвратом кредита в срок, поэтому сразу категоризируем данные по этому признаку. 

### Категоризация данных

**2.5.1 Категоризация по целям кредита**

In [12]:
# объявление функции categories_of_purposes_def() с параметром purpose, которая возвращает категорию цели кредита в зависимости от слов в purpose
def categories_of_purposes_def(purpose):
    if 'жил' in purpose or 'недвижим' in purpose:
        return 'недвижимость'
    if 'образован' in purpose:
        return 'образование'
    if 'автомобил' in purpose:
        return 'автомобиль'
    if 'свадьб' in purpose:
        return 'свадьба'
    
bc['categories_of_purposes'] = bc['purpose'].apply(categories_of_purposes_def) 
# создание нового столбца categories_of_purposes и сохранение в нём методом apply() результатов работы функции categories_of_purposes_def() для столбца purpose

Остальные вопросы исследования также требуют категоризации данных по разным признакам.

Для работой над [первым вопросом исследования](#1?) разделим всех клиентов банка по наличию/отсутствию детей.

**2.5.2 Категоризация по количеству детей** <a id='categories'></a>

In [13]:
bc['children'].unique() # получение уникальных значений столбца children

array([ 1,  0,  3,  2, -1,  4, 20,  5])

При знакомстве с уникальными значениями столбца `children` обнаружены артефакты - отрицательное значение (-1) и чрезмерно высокий показатель (20). Вероятнее всего, в значении "20" ошибочно указан лишний ноль. 

Исправим значения в колонке `children`:

In [14]:
print(bc[bc['children'] == -1]['children'].count() / len(bc))
bc[bc['children'] == 20]['children'].count() / len(bc)

0.0021907336627202388


0.003542462943973152

In [15]:
bc['children'] = bc['children'].abs() # получение модуля числа и сохранение в столбце children
bc['children'] = bc['children'].replace(20, 2) # замена в столбце children значений "20" на "2"
bc['children'].unique() # получение уникальных значений обновленного столбца children

array([1, 0, 3, 2, 4, 5])

Итак, среди клиентов банка есть:
- люди без детей,
- люди, имеющие одного или два ребёнка,
- люди, имеющие трёх или более детей (на территории России такие семьи считаются многодетными).

На эти категории и разделим клиентов банка:

In [16]:
# объявление функции having_children_def() с параметром children, которая возвращает категорию наличия/отсутствия детей в зависимости от количества детей 
def having_children_def(children):
    if children == 0:
        return 'бездетный'
    if children == 1 or children == 2:
        return 'один/два ребёнка'
    if children >= 3:
        return 'многодетный'
    
bc['having_children'] = bc['children'].apply(having_children_def)
# создание нового столбца having_children и сохранение в нём методом apply() результатов работы функции having_children_def() для столбца children

**2.5.3 Категоризация по семейному положению**

Чтобы ответить на [второй вопрос исследования](#2?), нам необходимо работать с семейным положением клиентов. Посмотрим, какие вообще статусы зафиксированы в нашей таблице в колонке `family_status`:

In [17]:
bc['family_status'].unique() # получение уникальных значений столбца family_status

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

Итак, клиентам банка в анкете предлагается пять вариантов определения своего семейного положения, но по большому счёту все они отображают только два свойства - состоит человек в браке или нет. Поэтому для упрощения анализа объединим клиентов в две большие группы:

In [18]:
# объявление функции having_family_def() с параметром status, которая возвращает категорию нахождения в браке в зависимости от количества семейного статуса 
def having_family_def(status):
    if status == 'женат / замужем':
        return 'состоит в браке'
    else:
        return 'не состоит в браке'
    
bc['having_family'] = bc['family_status'].apply(having_family_def)
# создание нового столбца having_family и сохранение в нём методом apply() результатов работы функции having_family_def() для столбца family_status

**2.5.4 Категоризация по уровню дохода**

Чтобы найти ответ на [третий вопрос исследования](#3?), нам необходимо разделить клиентов банка по уровню их достатка. Определим три группы ежемесячного дохода: средний, ниже среднего, высокий. 

Здесь необходимо отметить, что категоризация будет вестись только исходя из данных клиентов банка, а не согласно информации Росстата о средних доходах россиян или жителей конкретного населённого пункта.

In [19]:
# объявление функции monthly_income_def() с параметром income, которая возвращает категорию уровня дохода в зависимости от суммы ежемесячного дохода 
def monthly_income_def(income):
    if bc['total_income'].min() <= income < (bc['total_income'].min() + bc['total_income'].mean()) / 2:
        return 'ниже среднего'
    if (bc['total_income'].min() + bc['total_income'].mean()) / 2 <= income < (bc['total_income'].max() + bc['total_income'].mean()) / 2:
        return 'средний'
    if (bc['total_income'].max() + bc['total_income'].mean()) / 2 <= income <= bc['total_income'].max():
        return 'высокий'

bc['monthly_income'] = bc['total_income'].apply(monthly_income_def)
# создание нового столбца monthly_income и сохранение в нём методом apply() результатов работы функции monthly_income_def() для столбца total_income

**Вывод**

Мы категоризировали данные по нескольким разным признакам в зависимости от целей исследования и стоящими перед нами вопросами.

## Ответим на вопросы:

**- Есть ли зависимость между наличием детей и возвратом кредита в срок? <a id='1?'></a>**

In [20]:
bc['debt'].unique() # получение уникальных значений столбца debt

array([0, 1])

В колонке debt есть только два значения: "1" - если клиент имел задолженность по возврату кредитов; "0" - если задолженностей нет. Соответственно сумма значений в колонке debt равна количеству людей, имеющих долги.

[Категории по наличию/отсутствию детей](#categories) мы уже сформировали.
Выясним количество должников в каждой категории и определим их долю в категории.

In [21]:
bc.groupby('having_children')['debt'].sum() / bc.groupby('having_children')['children'].count()
# деление суммы значений debt в группировке по having_children на количество значений в той же группировке

having_children
бездетный           0.075438
многодетный         0.081579
один/два ребёнка    0.092654
dtype: float64

На первый взгляд непонятно: 7% и 9% - это мало или много? Посмотрим, какую долю все должники составляю от общего количества клиентов.

In [22]:
bc['debt'].sum() / len(bc) # деление суммы значений debt в таблице на длину всей таблицы

0.08115036822970076

**Вывод**

Среди клиентов, не имеющих детей, меньше всего должников по кредитам. 
У клиентов с одним или двумя детьми этот показатель выше, чем у остальных категорий. 
При этом среди многодетных клиентов количество людей с задолженностями на превышает общий показатель по всем клиентам, то есть находится в рамках "нормы".

**- Есть ли зависимость между семейным положением и возвратом кредита в срок? <a id='2?'></a>**

In [23]:
bc.groupby('having_family')['debt'].sum() / bc.groupby('having_family')['having_family'].count()
# деление суммы значений debt в группировке по having_family на количество значений в той же группировке

having_family
не состоит в браке    0.088865
состоит в браке       0.075452
dtype: float64

**Вывод**

Клиенты, состоящие в браке, имеют меньше задолженностей по кредитам, чем те, кто в браке не состоит.

**- Есть ли зависимость между уровнем дохода и возвратом кредита в срок? <a id='3?'></a>**

In [24]:
bc.groupby('monthly_income')['debt'].sum() / bc.groupby('monthly_income')['monthly_income'].count()
# деление суммы значений debt в группировке по monthly_income на количество значений в той же группировке

monthly_income
высокий          0.076923
ниже среднего    0.079343
средний          0.081525
dtype: float64

**Вывод**

Среди людей с разным уровнем дохода примерно равное количество клиентов, имеющих долги по кредитам - разница в долях весьма несущественна. Но клиенты с высоким уровнем ежемесячного дохода реже остальных допускают задолженности по кредитам. 

**- Как разные цели кредита влияют на его возврат в срок? <a id='4?'></a>**

In [25]:
bc.groupby('categories_of_purposes')['debt'].sum() / bc.groupby('categories_of_purposes')['categories_of_purposes'].count()
# деление суммы значений debt в группировке по categories_of_purposes на количество значений в той же группировке

categories_of_purposes
автомобиль      0.093590
недвижимость    0.072334
образование     0.092200
свадьба         0.080034
dtype: float64

**Вывод**

Клиенты, берущие кредит на покупку автомобиля, больше остальных имеют задолженности по кредитам. У тех, кто запрашивает кредит на оплату образования, этот показатель также высок. Меньше всего должников среди клиентов, которым кредит требуется для операций с недвижимостью.

## Общий вывод

Мы ответили на четыре вопроса и установили:

1. Зависимость между наличием детей и возвратом кредита в срок есть, но она не принципиальна: клиенты без детей действительно реже остальных пропускают платежи по кредитам, но при этом показатели многодетных клиентов ненамного хуже.

2. Выявлена зависимость между семейным положением и возвратом кредита в срок - клиенты, состоящие в браке, имеют меньше задолженностей по кредитам, чем те, кто в браке не состоит.

3. Зависимость между уровнем дохода и возвратом кредита в срок не имеет большого значения - клиенты с разным уровнем ежемесячного дохода примерно в равной степени допускают задолженности по кредитам.

4. Обнаружена зависимость между целями кредита и возвратом кредита в срок: при покупке автомобиля или оплате образования чаще всего нарушаются сроки оплаты кредита; самые дисциплинированные - клиенты, запрашивающие кредит на операции с недвижимостью.

**Резюмируя всё вышесказанное, "идеальный" клиент банка - бездетный семьянин с высоким уровнем дохода, запрашивающий кредит на недвижимость.**